In [1]:
import cv2
import mediapipe as mp
import json

# Initialize pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)
mp_drawing = mp.solutions.drawing_utils

# Load and process the reference image
image = cv2.imread('trianglepose.jpg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
result = pose.process(image_rgb)

# Extract keypoints and save them
if result.pose_landmarks:
    keypoints = [{'x': lm.x, 'y': lm.y, 'z': lm.z, 'visibility': lm.visibility} for lm in result.pose_landmarks.landmark]
    with open('reference_keypoints.json', 'w') as f:
        json.dump(keypoints, f)
    print("Keypoints saved successfully.")
else:
    print("No pose detected.")


Keypoints saved successfully.


C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [2]:
import cv2
import mediapipe as mp
import json
import math
import pyttsx3

# Load reference keypoints (example JSON file containing reference points)
with open('reference_keypoints.json', 'r') as f:
    reference_keypoints = json.load(f)

# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    return math.sqrt((point1['x'] - point2['x'])**2 + (point1['y'] - point2['y'])**2)

# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
cap = cv2.VideoCapture(0)

# Initialize the pyttsx3 engine for text-to-speech
engine = pyttsx3.init()

# Function to give audio feedback
def give_audio_feedback(message):
    engine.say(message)
    engine.runAndWait()

# Function to show the result in a separate window
def show_result(frame, pose_status, color=(0, 255, 0), similarity=None):
    # Display similarity if available
    if similarity is not None:
        accuracy_text = f"Accuracy: {similarity:.2f}%"
        cv2.putText(frame, accuracy_text, (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.putText(frame, pose_status, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.imshow('Pose Detection', frame)  # This will display the frame in a separate window

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert frame to RGB (required by MediaPipe)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(frame_rgb)
        
        if result.pose_landmarks:
            # Get the keypoints of the detected pose
            current_keypoints = [{'x': lm.x, 'y': lm.y, 'z': lm.z, 'visibility': lm.visibility} for lm in result.pose_landmarks.landmark]
            
            # Calculate similarity score between the current pose and reference pose
            match_count = 0
            for ref, cur in zip(reference_keypoints, current_keypoints):
                distance = calculate_distance(ref, cur)
                if distance < 0.1:  # Adjust threshold for similarity as needed
                    match_count += 1
            
            similarity = (match_count / len(reference_keypoints)) * 100
            pose_status = 'Pose Correct' if similarity > 70 else 'Pose Incorrect'
            
            # Green if correct pose, Red if incorrect pose
            color = (0, 255, 0) if similarity > 70 else (0, 0, 255)
            
            # Give audio feedback only when the pose is correct
            if similarity > 70:
                give_audio_feedback("You are doing great!")
            
            # Show the result (pose status) and accuracy in the separate window
            show_result(frame, pose_status, color, similarity)

        # Break the loop if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Release the camera and close the window
    cap.release()
    cv2.destroyAllWindows()
    print("Camera released and window closed.")


Camera released and window closed.
